In [1]:
import torch
import torch.nn as nn
import math
from tokenizers import Tokenizer
import re

# Định nghĩa lại các class và function cần thiết
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class LucBatTransformer(nn.Module):
    def __init__(self, vocab_size: int, d_model: int = 512, nhead: int = 8, 
                 num_layers: int = 6, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, 
                                                  dim_feedforward=2048, 
                                                  dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.embedding(src) * math.sqrt(self.d_model)
        embedded = self.pos_encoder(embedded)
        output = self.transformer_encoder(embedded)
        output = self.dropout(output)
        output = self.fc_out(output)
        return output

def get_rhyme(word):
    if not word or not is_vietnamese_word(word):
        return ''
    vowels = 'aàảãáạăằẳẵắặâầẩẫấậeèẻẽéẹêềểễếệiìỉĩíịoòỏõóọôồổỗốộơờởỡớợuùủũúụưừửữứựyỳỷỹýỵ'
    word = word.lower()
    rhyme = ''
    for i in range(len(word)-1, -1, -1):
        if word[i] in vowels:
            rhyme = word[i:]
            break
    return rhyme

def is_vietnamese_word(word):
    vietnamese_pattern = r'^[aàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnoòỏõóọôồổỗốộơờởỡớợpqrstuùủũúụưừửữứựvwxyỳỷỹýỵz]+$'
    return bool(re.match(vietnamese_pattern, word.lower()))

def get_tone_type(word):
    vowels = {
        'even': ['a', 'ă', 'â', 'e', 'ê', 'o', 'ô', 'ơ', 'i', 'u', 'ư', 'y',
                'à', 'ằ', 'ầ', 'è', 'ề', 'ò', 'ồ', 'ờ', 'ì', 'ù', 'ừ', 'ỳ'],
        'uneven': ['á', 'ắ', 'ấ', 'é', 'ế', 'ó', 'ố', 'ớ', 'í', 'ú', 'ứ', 'ý',
                   'ạ', 'ặ', 'ậ', 'ẹ', 'ệ', 'ọ', 'ộ', 'ợ', 'ị', 'ụ', 'ự', 'ỵ',
                   'ả', 'ẳ', 'ẩ', 'ẻ', 'ể', 'ỏ', 'ổ', 'ở', 'ỉ', 'ủ', 'ử', 'ỷ',
                   'ã', 'ẵ', 'ẫ', 'ẽ', 'ễ', 'õ', 'ỗ', 'ỡ', 'ĩ', 'ũ', 'ữ', 'ỹ']
    }
    
    for char in word.lower():
        if char in vowels['even']:
            return 'even'
        elif char in vowels['uneven']:
            return 'uneven'
    return 'even'

def get_specific_tone(word):
    huyen_vowels = ['à', 'ằ', 'ầ', 'è', 'ề', 'ò', 'ồ', 'ờ', 'ì', 'ù', 'ừ', 'ỳ']
    khong_dau_vowels = ['a', 'ă', 'â', 'e', 'ê', 'o', 'ô', 'ơ', 'i', 'u', 'ư', 'y']
    
    for char in word.lower():
        if char in huyen_vowels:
            return 'huyen'
        elif char in khong_dau_vowels:
            return 'ngang'
    return None

def check_rhyme(word1, word2):
    rhyme1 = get_rhyme(word1)
    rhyme2 = get_rhyme(word2)
    return rhyme1 == rhyme2

def generate_lucbat(model, tokenizer, prompt, temperature=0.7, device='cpu'):
    def get_valid_word(candidates, position, line_type, prev_line=None, prev_rhyme=None):
        # Quy tắc thanh điệu cho câu lục và câu bát
        tone_rules = {
            'luc': {
                2: 'even',    # Chữ 2: thanh bằng
                4: 'uneven',  # Chữ 4: thanh trắc
                6: 'even'     # Chữ 6: thanh bằng
            },
            'bat': {
                2: 'even',    # Chữ 2: thanh bằng
                4: 'uneven',  # Chữ 4: thanh trắc
                6: 'even',    # Chữ 6: thanh bằng
                8: 'even'     # Chữ 8: thanh bằng
            }
        }
        
        for word in candidates:
            if not isinstance(word, str) or len(word.strip()) == 0:
                continue
                
            word_tone = get_tone_type(word)
            
            # Kiểm tra quy tắc thanh điệu theo vị trí
            if position in tone_rules[line_type]:
                if word_tone != tone_rules[line_type][position]:
                    continue
            
            # Kiểm tra quy tắc vần
            if line_type == 'luc' and position == 6:
                # Chữ cuối câu lục phải vần với chữ 6 câu bát tiếp theo
                if prev_rhyme and not check_rhyme(word, prev_rhyme):
                    continue
                    
            if line_type == 'bat':
                if position == 6:
                    # Chữ 6 câu bát phải vần với chữ cuối câu lục trước
                    if prev_line and not check_rhyme(word, prev_line.split()[-1]):
                        continue
                elif position == 8:
                    # Quy tắc đặc biệt cho chữ 6 và 8 của câu bát
                    word6 = prev_line.split()[5] if prev_line else None
                    if word6:
                        word6_tone = get_specific_tone(word6)
                        word8_tone = get_specific_tone(word)
                        
                        # Nếu chữ 6 thanh ngang thì chữ 8 phải thanh huyền và ngược lại
                        if word6_tone == 'ngang' and word8_tone != 'huyen':
                            continue
                        if word6_tone == 'huyen' and word8_tone != 'ngang':
                            continue
            
            return word
        
        return candidates[0]  # Trường hợp không tìm được từ phù hợp

    model.eval()
    lines = []
    current_line = prompt.split()
    prev_rhyme = None
    
    # Sinh 4 dòng thơ
    for i in range(4):
        line_type = 'bat' if i % 2 == 1 else 'luc'
        target_length = 8 if line_type == 'bat' else 6
        
        while len(current_line) < target_length:
            input_text = ' '.join(lines + [' '.join(current_line)])
            input_ids = torch.tensor(tokenizer.encode(input_text).ids).unsqueeze(0).to(device)
            
            with torch.no_grad():
                outputs = model(input_ids)
                next_token_logits = outputs[0, -1, :] / temperature
                
                # Lấy nhiều candidates hơn để có nhiều lựa chọn
                top_k = 100
                top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)
                probs = torch.softmax(top_k_logits, dim=-1)
                
                candidate_indices = top_k_indices[torch.multinomial(probs, num_samples=20)]
                candidates = [tokenizer.decode([idx.item()]).strip() for idx in candidate_indices]
                
                next_word = get_valid_word(
                    candidates,
                    len(current_line) + 1,
                    line_type,
                    prev_line=' '.join(current_line) if current_line else None,
                    prev_rhyme=prev_rhyme
                )
                
                if next_word:
                    current_line.append(next_word)
        
        lines.append(' '.join(current_line))
        if line_type == 'bat':
            prev_rhyme = current_line[5]  # Lưu chữ thứ 6 của câu bát
        current_line = []
    
    return '\n'.join(lines)

# Hàm chính để load model và sinh thơ
def load_model_and_generate(model_path, tokenizer_path, prompt):
    # Load tokenizer
    tokenizer = Tokenizer.from_file(tokenizer_path)
    
    # Khởi tạo model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LucBatTransformer(
        vocab_size=tokenizer.get_vocab_size(),
        d_model=512,
        nhead=8,
        num_layers=6
    ).to(device)
    
    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
    model.eval()
    
    # Sinh thơ
    poem = generate_lucbat(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        temperature=0.8,
        device=device
    )
    
    return poem

def print_model_dimensions():
    # Các thông số cố định của mô hình
    vocab_size = 12258  # Kích thước từ điển
    d_model = 512      # Kích thước embedding
    nhead = 8          # Số attention heads
    num_layers = 6     # Số layers
    dropout = 0.1      # Tỷ lệ dropout
    
    # Khởi tạo model
    model = LucBatTransformer(
        vocab_size=vocab_size,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    )
    
    print("\n=== Kích thước của mô hình ===")
    print(f"Vocabulary Size: {vocab_size:,} tokens")
    print(f"Embedding Dimension (d_model): {d_model}")
    print(f"Number of Attention Heads: {nhead}")
    print(f"Number of Layers: {num_layers}")
    print(f"Feed Forward Dimension: 2048")  # Giá trị mặc định trong mô hình
    print(f"Dropout Rate: {dropout}")
    
    # Tính tổng số tham số
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\nTổng số tham số: {total_params:,}")
    
    print("\n=== Kích thước đầu vào ===")
    print("Input shape: [batch_size, sequence_length]")
    print("- Batch size: 16 (mặc định)")
    print("- Max sequence length: 128 tokens")
    print("\nEmbedding output shape: [batch_size, sequence_length, d_model]")
    print(f"- [16, 128, {d_model}]")
    
    print("\n=== Chi tiết các layer ===")
    print("1. Embedding Layer:")
    print(f"   Input: [batch_size, sequence_length]")
    print(f"   Output: [batch_size, sequence_length, {d_model}]")
    
    print("\n2. Positional Encoding:")
    print(f"   Input: [batch_size, sequence_length, {d_model}]")
    print(f"   Output: [batch_size, sequence_length, {d_model}]")
    
    print("\n3. Transformer Encoder:")
    print(f"   - Number of layers: {num_layers}")
    print(f"   - Attention heads per layer: {nhead}")
    print(f"   - Feed-forward dimension: 2048")
    print(f"   Input: [batch_size, sequence_length, {d_model}]")
    print(f"   Output: [batch_size, sequence_length, {d_model}]")
    
    print("\n4. Output Layer:")
    print(f"   Input: [batch_size, sequence_length, {d_model}]")
    print(f"   Output: [batch_size, sequence_length, {vocab_size}]")


# Sử dụng
if __name__ == "__main__":
    model_path = "model/lucbat_model.pth"  # Đường dẫn tới file model
    tokenizer_path = "model/tokenizer.json"  # Đường dẫn tới file tokenizer
    
    # In kích thước mô hình
    print_model_dimensions()
    
    # Sinh thơ với prompt
    prompt = "Mùa xuân"
    poem = load_model_and_generate(model_path, tokenizer_path, prompt)
    print("\n=== Bài thơ được sinh ra ===")
    print(poem)



c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")



=== Kích thước của mô hình ===
Vocabulary Size: 12,258 tokens
Embedding Dimension (d_model): 512
Number of Attention Heads: 8
Number of Layers: 6
Feed Forward Dimension: 2048
Dropout Rate: 0.1

Tổng số tham số: 31,478,754

=== Kích thước đầu vào ===
Input shape: [batch_size, sequence_length]
- Batch size: 16 (mặc định)
- Max sequence length: 128 tokens

Embedding output shape: [batch_size, sequence_length, d_model]
- [16, 128, 512]

=== Chi tiết các layer ===
1. Embedding Layer:
   Input: [batch_size, sequence_length]
   Output: [batch_size, sequence_length, 512]

2. Positional Encoding:
   Input: [batch_size, sequence_length, 512]
   Output: [batch_size, sequence_length, 512]

3. Transformer Encoder:
   - Number of layers: 6
   - Attention heads per layer: 8
   - Feed-forward dimension: 2048
   Input: [batch_size, sequence_length, 512]
   Output: [batch_size, sequence_length, 512]

4. Output Layer:
   Input: [batch_size, sequence_length, 512]
   Output: [batch_size, sequence_length, 